## Purpose: 
This Notebook is designed as an attempt to implement the Gollini 2014 method of extracting latent positions from (multiple) network view(s).

The method implements a EM - algorithm in combination with a Variational methods approach to estimation. 

The impact of this is twofold: 

Firstly, The speed gain should be significant in comparison to alternative (bayesian) methods. 

Secondly, The extraction of latent positions should be beneficial in relation to controlling for selection effects. 


## The algorithm: 

The algorithm consist of two steps: The E and the M step. Overall they are both optimization steps in the variational methods framework.

#### E - Step:  estimate z_i and Sigma_i 

#### M - Step: Estimate alpha_M_i  and alpha_V_i^2


    

This Step designs a function that can estimate z_i in the single use case Next we will attempt a vectorized approach


In [2]:
from numpy.linalg import norm, det
from numpy.random import multivariate_normal
import numpy as np
from numpy.random import uniform, normal
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
import timeit
import seaborn as sns
import pandas as pd

In [3]:
## Simulate companion matrix: 


def Companion_matrix(Z, alpha_0, alpha_1, N, threshold):
    
    dist_c = squareform(pdist(Z))
    
    dist_c = dist_c*dist_c

    # latent utility nxn matrix:

    alpha_0_nxn = np.ones((N,N))*alpha_0

    alpha_1_nxn = dist_c*alpha_1

    # noise term: - random gaussian noise where the upper diagonal is taken and transposed and the diagonal is divided by two
    #E_nxn = normal(size=(N,N))
    E_nxn = np.triu(normal(size=(N,N)))
    E_nxn = E_nxn + E_nxn.T
    np.fill_diagonal(E_nxn,0)

    latent_utility = alpha_0_nxn + alpha_1_nxn + E_nxn

    return np.where(latent_utility > threshold,1,0)


In [221]:
# parameters for testing: 
D=4
N = 10

# Setting the true mean and cov-var of Z
mean_z = np.zeros(D)
cov_z  = np.identity(D)
alpha_0 = 0
alpha_1 = -0.3

# generating Z:

Z = multivariate_normal(mean_z, cov_z, N)

# Companion matrix - This function is a bit unstable and needs to be better constructed - it will do the trick though

Y = Companion_matrix(Z,  alpha_0, alpha_1, N,  0)

print("Z dim:", np.shape(Z))
print("Y dim:", np.shape(Y))
print(Y)

Z dim: (10, 4)
Y dim: (10, 10)
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]]


In [222]:
#prior VARIANCE
alpha_v_prior = 1
alpha_m_prior = 0


# this function should produce a gradient of size D*1:
# for N = 1000 this precedure takes roughly 3 seconds: If this proves to be too slow one can consider working with tensors
def zi_Gradient(i, D, sigma_est, Z_1, alpha_m, alpha_v):
    
    from numpy.linalg import inv as inv
    
    #error handling
    if D != np.shape(sigma_est)[0]:
        raise ValueError("dimension of sigma_est is not equal to dimensionality D")

    # (I + 4sigma)^-1 DxD matrix
    alpha=np.identity(D) +  4*sigma_est
    alpha_inv = inv(alpha)
    
    # Z_ij = (zi - zj) (N-1)xD matrix (i excluded)
    z_i= Z_1[i,:]
    Z_j = np.delete(Z_1, (i), axis=0)
    Z_ij = z_i - Z_j
    
    
    # c *exp(z_ij*alpha_inv*z_ij^T) = c*exp(quadform)
    
    c = np.sqrt(det(alpha))/np.exp(alpha_m + 0.5*alpha_v)
        
    def quadform(Z_ij, alpha_inv):
        ZalphaZT = np.zeros(np.shape(Z_ij)[0])
        for j in range(np.shape(Z_ij)[0]):
             ZalphaZT[j]= Z_ij[j,:].dot(alpha_inv).dot(Z_ij[j,:].T)
        return ZalphaZT
        
    
    exp_quadform = np.exp(quadform(Z_ij, alpha_inv))
    
    
    
    # beta = [1 + c*exp(quadform)^-1] (N-1)*1 matrix
    
    beta = 1/(1 + c*exp_quadform)
    
    # Sum_j (z_i - z_j)*beta (N-1)*D matrix
        
    Sum_j = (Z_ij.T * beta).T
    
    # Next we want to get alpha_inv * sum over N dimension to get a D*1
    
    sum_j = np.sum(Sum_j,axis=0)
    
    # returns the gradient
    
    return -2*alpha_inv.dot(sum_j)

zi_Gradient(1, D, cov_z, Z, alpha_m_prior, alpha_v_prior)



array([-0.01378887, -0.04014499,  0.05841005, -0.03253872])

In [239]:
# This function should produce a Hessian of size DxD
# -2*alpha_inv*sum_j(1 + c*exp(quadform))^-1 * [I - Z_ij Z_ij^T*alpha_inv/[1+ c*exp(quadform)]]
def zi_Hessian(i, D, sigma_est, Z_1, alpha_m, alpha_v):
    from numpy.linalg import inv as inv
    # The Hessian is composed of alpha*beta*gamma (DxD * 1x1 * DxD)
    
    # alpha - part: DxD matrix
    # (I + 4sigma)^-1 DxD matrix
    alpha=np.identity(D) +  4*sigma_est
    alpha_inv = inv(alpha)
    
    # beta  - part: (N-1)x1
    # Z_ij = (zi - zj) (N-1)xD matrix (i excluded)
    z_i= Z_1[i,:]
    Z_j = np.delete(Z_1, (i), axis=0)
    Z_ij = z_i - Z_j
    
    
    # c *exp(z_ij*alpha_inv*z_ij^T) = c*exp(quadform)
    
    c = np.sqrt(det(alpha))/np.exp(alpha_m + 0.5*alpha_v)
        
    def quadform(Z_ij, alpha_inv):
        ZalphaZT = np.zeros(np.shape(Z_ij)[0])
        for j in range(np.shape(Z_ij)[0]):
             ZalphaZT[j]= Z_ij[j,:].dot(alpha_inv).dot(Z_ij[j,:].T)
        return ZalphaZT
        
    
    exp_quadform = np.exp(quadform(Z_ij, alpha_inv))
    
    # the beta part (N-1)*1
    beta = 1/(1 + c*exp_quadform)
    
    #now we broadcast beta into a (N-1)xDxD matrix where DxD consist of ones*beta[i]
    N = np.shape(beta)[0]
    def broadc_NDD(beta,D):
        # This function broadcasts from a (N-1)x1 --> (N-1)xDxD (where DxD(i) = Ones*beta[i])
        return np.reshape(np.repeat([np.repeat(beta,D)],D, axis=0).T,(N,D,D))
    
    beta_NDD = broadc_NDD(beta,D)
    
    # gamma = = 1/num*denom = 1/num * Z_ijZ_ij.T * alpha_inv (1x1)
    
    # Z_ij * Z_ij.T = Dx1 1xD = DxD 
    
    Zij=np.reshape(Z_ij,(shape(Z_ij)[0],D,1))
    ZijT=np.reshape(Z_ij,(shape(Z_ij)[0],1,D))
    
    denom = 2*(Zij * ZijT).dot(alpha_inv)
    num = np.reshape(1 + 1/c * 1/exp_quadform,(9,1,1))
    identity = np.identity(D)
    
    gamma_NDD = identity - denom * num
    
    #Hessian = -2 alpha_inv * sum(gamma_NDD * beta_NDD)
    
    Hessian =-2*alpha_inv.dot(np.sum(gamma_NDD * beta_NDD, axis = 0))
    
    return Hessian
    
    
zi_Hessian(1, D, cov_z, Z, alpha_m_prior, alpha_v_prior)


array([[-0.04170633,  0.00996534, -0.01208541,  0.00691196],
       [ 0.00996534, -0.04593376, -0.01407383,  0.00778572],
       [-0.01208541, -0.01407383, -0.01261962, -0.01003679],
       [ 0.00691196,  0.00778572, -0.01003679, -0.04085983]])

In [ ]:
## from numpy import shape
N=3
D=2

a = np.array([[[1,2],[3,4]],[[1,2],[3,4]],[[1,2],[3,4]]])
b = np.array([1,2,3])

test = np.repeat(b,2)
test = np.repeat([test],2, axis=0)
test2 = np.reshape(test.T,(3,2,2))
print(shape(test2))
print(test2)
print(a)

In [113]:
a - test2

array([[[ 0,  1],
        [ 2,  3]],

       [[-1,  0],
        [ 1,  2]],

       [[-2, -1],
        [ 0,  1]]])

[[[1 1]
  [1 1]]

 [[2 2]
  [2 2]]

 [[3 3]
  [3 3]]] [[[1 2]
  [3 4]]

 [[1 2]
  [3 4]]

 [[1 2]
  [3 4]]]


In [133]:
a = np.array([[1,2,3], [4,6,7]])
b= np.reshape(a,(2,1,3))
c = np.reshape(a.T,(2,1,3))
print(b)
print(c)

[[[1 2 3]]

 [[4 6 7]]]
[[[1 4 2]]

 [[6 3 7]]]


In [169]:
a = np.array([[1,2],[2,3]])
b = np.array([[2,3],[4,6]])
print(a*b)
print(a.dot(b))

[[ 2  6]
 [ 8 18]]
[[10 15]
 [16 24]]
